In [ ]:
from qiskit import QuantumCircuit, IBMQ, execute
from qiskit.circuit import QuantumRegister
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.tools.monitor import job_monitor

def bv_oracle(secret_string):
    s = secret_string[::-1]
    secret_string_length = len(secret_string)
    qc = QuantumCircuit(secret_string_length + 1, name='oracle')
    for i in range(secret_string_length):
        if s[i] == '1':
            qc.cx(i, secret_string_length)
    return qc

def bv_circuit(n, oracle):
    qc = QuantumCircuit(n+1, n)
    qc.h(range(n+1))
    qc.z(n)
    qc.append(oracle, [0,1,2,3])
    qc.h(range(n))    
    qc.measure(range(n), range(n))
    return qc

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education')
bogota = provider.get_backend('ibmq_bogota')

SECRET_STRING = '011'
N = len(SECRET_STRING)
oracle011 = bv_oracle(SECRET_STRING)
oracle_breaker_qc = bv_circuit(N, oracle011)

layout = [3,1,0,2]
calibration_layout = [3,1,0]
qr = QuantumRegister(3)
qubit_list = [0,1,2]
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='mcal')

job = execute(oracle_breaker_qc, bogota)
BOGOTA_NAIVE_JOB_ID = job.job_id()
print('\n'.join(['---------------------------------------------', f'BOGOTA_NAIVE_JOB_ID: {BOGOTA_NAIVE_JOB_ID}', '---------------------------------------------']))
job_monitor(job)

print('\n')

job = execute(meas_calibs, bogota, shots=8192, initial_layout=calibration_layout)
BOGOTA_MEASCAL_JOB_ID = job.job_id()
print('\n'.join(['-----------------------------------------------', f'BOGOTA_MEASCAL_JOB_ID: {BOGOTA_MEASCAL_JOB_ID}', '-----------------------------------------------']))
job_monitor(job)

print('\n')

job = execute(oracle_breaker_qc, bogota, initial_layout=layout)
BOGOTA_EDUCATED_JOB_ID = job.job_id()
print('\n'.join(['------------------------------------------------', f'BOGOTA_EDUCATED_JOB_ID: {BOGOTA_EDUCATED_JOB_ID}', '------------------------------------------------']))
job_monitor(job)